In [1]:
# Created by Frederic on 15.03.2025.
# 
# This script is used to create a HashMap out of the binary data of the traffic simulation.
#
# Steps:
# - Read the binary data from a duckdb file
# - Split the data into the different segments of the roads
# - Extract the density by averaging over the last dt time steps (dt*10 seconds)
# - Use these information to create random vehicles
#        - In this step handle the min distance issue to avoid disappearing vehicles
# - Create a HashMap with the information of the vehicles (parameters and position)

In [2]:
import duckdb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import Hashmap_Creation as hc

In [3]:
# First we create a dataframe and csv with all possible vehicle types by reading a .rou.xml file
import xml.etree.ElementTree as ET

tree = ET.parse('../../../DemoCase/Routes2000_2_Krauss_represent_traffic_BBV_30.rou.xml')

root = tree.getroot()

# We are only interested in the vehicle types parameters
vehicle_types = root.findall('vType')

# Read the content of the vehicle types: lcStrategic, tau, id, accel, decel, sigma, length, width, maxSpeed, vClass, carFollowModel, color, guiShape

vehicle_types_dict = {}
for vehicle_type in vehicle_types:
    vehicle_types_dict[vehicle_type.attrib['id']] = vehicle_type.attrib
    
vehicle_types_df = pd.DataFrame(vehicle_types_dict).T

# Set the id as index
vehicle_types_df = vehicle_types_df.set_index('id')

vehicle_types_df.to_csv('vehicle_types.csv') 

In [4]:
# Open the duckdb file
path = "D:/extern3/Traffic_KITracking/scenario.duckdb"

con = duckdb.connect(path)

In [5]:
lanes = 2

time_steps_before_simulation = 100
sensors_per_segment = 20
length_per_sensor = 5

tstart = 0
tend = 16000
twin = 1
tstep = 1

xstart = 0
xend = 4000

x = hc.selectx('rawdata', lanes, tstart, tstart+time_steps_before_simulation, xstart, xstart + xend, con)
# Only keep the first 2 of the second dimension
x = x[:, :2, :]
print(x.shape)
# Split the data into the different segments of the roads
x_new = np.zeros([x.shape[0], x.shape[1], x.shape[2]//sensors_per_segment, sensors_per_segment])

for time_step in range(x.shape[0]):
    for lane in range(x.shape[1]):
        x_new[time_step, lane] = np.array(np.split(x[time_step, lane], x.shape[2]//sensors_per_segment), dtype=np.uint8)
print(x_new.shape)

density = np.zeros([x_new.shape[0], x_new.shape[1], x_new.shape[2]])
for time_step in range(x_new.shape[0]):
    for lane in range(x_new.shape[1]):
        density[time_step, lane] = np.mean(x_new[time_step, lane], axis=1)
        
print(density.shape)

density = np.mean(density, axis=0)

print(density.shape)

density_per_segment = np.mean(density, axis=0)

print(density_per_segment.shape)

(100, 2, 4000)
(100, 2, 200, 20)
(100, 2, 200)
(2, 200)
(200,)


In [6]:
x = hc.selectx('rawdata', lanes, tstart, tstart+time_steps_before_simulation, xstart, xstart + xend, con)
x = hc.binary_data_to_segments_density(x)
x[:10]


array([0.1355 , 0.13675, 0.1345 , 0.1495 , 0.1415 , 0.106  , 0.1135 ,
       0.15525, 0.17525, 0.12   ])

In [7]:
lane_elements_per_segment = 2
possible_elements = []
for i in range(density_per_segment.shape[0]):
    possible_elements.append([i*2, i*2+1])
    
len(possible_elements)

200

In [8]:
vehicle_dict, route_dict = hc.randomly_place_vehicle(vehicle_types_df, density_per_segment[10], 100, possible_elements[10])

vehicle_dict

No vehicles fit in the available space
No more vehicles can be placed


{'CC1_0': 'CC1', 'SPV2_0': 'SPV2'}

In [9]:
hc.create_rou_xml(vehicle_dict, route_dict, 'Template.rou.xml')

Created file routes/test5.rou.xml


In [10]:
x = hc.selectx('rawdata', lanes, tstart, tstart+time_steps_before_simulation, xstart, xstart + xend, con)
x = hc.binary_data_to_segments_density(x)
vehicle_dict, route_dict = hc.randomly_fill_road(vehicle_types_df, x, possible_elements, 100)

No vehicles fit in the available space
No more vehicles can be placed
No vehicles fit in the available space
No more vehicles can be placed
No vehicles fit in the available space
No more vehicles can be placed
No vehicles fit in the available space
No more vehicles can be placed
No vehicles fit in the available space
No more vehicles can be placed
No vehicles fit in the available space
No more vehicles can be placed
No vehicles fit in the available space
No more vehicles can be placed
No vehicles fit in the available space
No more vehicles can be placed
No vehicles fit in the available space
No more vehicles can be placed
No vehicles fit in the available space
No more vehicles can be placed
No vehicles fit in the available space
No more vehicles can be placed
No vehicles fit in the available space
No more vehicles can be placed
No vehicles fit in the available space
No more vehicles can be placed
No vehicles fit in the available space
No more vehicles can be placed
No vehicles fit in t

In [15]:
hc.create_rou_xml(vehicle_dict, route_dict, 'Template.rou.xml')

Created file routes/test.rou.xml
